<a href="https://colab.research.google.com/github/ejf78/cdc_vitalsigns/blob/master/Vital%20Signs%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compile and Anlayse Vital Signs


After a little bit of manual cleaning, I got the historic data (2000-2010) for many indicators into a common format, stored in several different excel files with several sheets each. I also saved the modern (2010 onward) indicators as CSVs, to avoid re-pulling from the APIs. In this notebook, I'll compile everything and set up some analysis. 

### Colab-Specific Steps

In [1]:
# clone the github respository, so that we have all the necessary files 
!git clone https://github.com/ejf78/cdc_vitalsigns.git

Cloning into 'cdc_vitalsigns'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 43 (delta 9), reused 36 (delta 5), pack-reused 0
Unpacking objects: 100% (43/43), done.
Checking out files: 100% (33/33), done.


In [2]:
# install geopandas in google colab 
!pip install geopandas

     |████████████████████████████████| 1.0 MB 23.9 MB/s 
     |████████████████████████████████| 6.3 MB 54.5 MB/s 
     |████████████████████████████████| 15.4 MB 29.4 MB/s 


In [3]:
# load packages
import pandas as pd
import os # for navigating directories
import requests
import geopandas as gpd
from geopandas import GeoDataFrame

In [4]:
# navigate into the directory
os.chdir("cdc_vitalsigns")

## Load and Compile Data 

In [128]:
# api info 
# read list of indicators 
api_df = pd.read_csv("VS-Indicator-APIs_EF.csv") # new version - I've labeled which API calls to make under 'pull'
api_df.set_index("ShortName", inplace=True, drop = False) # drop = False I want ShortName as a column 
# add a column for indicator shortname, without the XX's, and lowercase for uniformity 
api_df["shortname_noX"] = api_df.ShortName.str.replace("XX","").str.lower()
api_df

,Indicator Number,Indicator,ShortName,Section,API,pull,shortname_noX
ShortName,,,,,,,
tpopXX,1,Total Population,tpopXX,Census Demographics,https://services1.arcgis.com/mVFRs7NF4iFitgbY/...,1,tpop
maleXX,2,Total Male Population,maleXX,Census Demographics,https://services1.arcgis.com/mVFRs7NF4iFitgbY/...,1,male
femaleXX,3,Total Female Population,femaleXX,Census Demographics,https://services1.arcgis.com/mVFRs7NF4iFitgbY/...,1,female
paaXX,4,Percent of Residents - Black/African-American ...,paaXX,Census Demographics,https://services1.arcgis.com/mVFRs7NF4iFitgbY/...,1,paa
pwhiteXX,5,Percent of Residents - White/Caucasian (Non-Hi...,pwhiteXX,Census Demographics,https://services1.arcgis.com/mVFRs7NF4iFitgbY/...,1,pwhite
...,...,...,...,...,...,...,...
pread8XX,211,Percentage of 8th Grade Students who Met or Ex...,pread8XX,Education and Youth,https://services1.arcgis.com/mVFRs7NF4iFitgbY/...,1,pread8
palg1XX,212,Percentage of Students who Met or Exceeded PAR...,palg1XX,Education and Youth,https://services1.arcgis.com/mVFRs7NF4iFitgbY/...,1,palg1
palg2XX,213,Percentage of Students who Met or Exceeded PAR...,palg2XX,Education and Youth,https://services1.arcgis.com/mVFRs7NF4iFitgbY/...,1,palg2


#### Historic Data (2000 - 2006) 

Already cleaned 

In [37]:
### historic data 
# read from CSV 
historic_indicators = pd.read_csv("precompiled_historic_indicators.csv")
# pivot longer for join 
hvs = historic_indicators.melt(id_vars = ["CSA", "indicator", "indicator_category"], 
                        var_name = "year")
# drop indicator category (will add it later so that it's uniform)
hvs = hvs.drop(["indicator_category"], axis = 1)
# remove the XX's from the indicator names 
hvs["indicator"] = hvs.indicator.str.replace("XX","")

#### Clean data from 2010 onward

In [94]:
### modern data 
# read in data files for modern indicators (saved from previous API kills)
mvs1 = pd.read_csv("modern_vital_signs_raw_1.csv")
mvs2 = pd.read_csv("modern_vital_signs_raw_2.csv")
mvs3 = pd.read_csv("modern_vital_signs_raw_3.csv")



## reformat / melt 
# msv1 
objectid_cols = [col for col in mvs1.columns if "OBJECTID" in col] # remove any columns called OBJECTID
mvs1 = mvs1.drop(objectid_cols, axis = 1) 
# drop geometry as well; it's causing some problems
mvs1 = mvs1.drop(['Shape__Area', 'Shape__Length', "geometry"], axis = 1)
index_cols = ["CSA2010"]
mvs_df_1 = mvs1.melt(id_vars = index_cols,
                     var_name = "year-indicator",
                     value_name = "value")
# msv2 
objectid_cols = [col for col in mvs2.columns if "OBJECTID" in col]
mvs2 = mvs2.drop(objectid_cols, axis = 1)
mvs2 = mvs2.drop(['Shape__Area', 'Shape__Length', "geometry"], axis = 1)
mvs_df_2 = mvs2.melt(id_vars = index_cols,
                     var_name = "year-indicator",
                     value_name = "value")
# msv3
objectid_cols = [col for col in mvs3.columns if "OBJECTID" in col]
mvs3 = mvs3.drop(objectid_cols, axis = 1)
mvs3 = mvs3.drop(['Shape__Area', 'Shape__Length', "geometry","CSA2010_1"], axis = 1)  # one off issue: drop "CSA2010_1" column
mvs_df_3 = mvs3.melt(id_vars = index_cols,
                     var_name = "year-indicator",
                     value_name = "value")

# rename
mvs_df_1 = mvs_df_1.rename(columns = {"CSA2010":"CSA"})
mvs_df_2 = mvs_df_2.rename(columns = {"CSA2010":"CSA"})
mvs_df_3 = mvs_df_3.rename(columns = {"CSA2010":"CSA"})
## add column for year, based on indicator/year field 
mvs_df_1["year"] = ['20' + i[-2:] for i in mvs_df_1["year-indicator"]]
mvs_df_2["year"] = ['20' + i[-2:] for i in mvs_df_2["year-indicator"]]
mvs_df_3["year"] = ['20' + i[-2:] for i in mvs_df_3["year-indicator"]]
## add column for indicator, based on indicator/year field 
mvs_df_1["indicator"] = [i[:-2] for i in mvs_df_1["year-indicator"]]
mvs_df_2["indicator"] = [i[:-2] for i in mvs_df_2["year-indicator"]]
mvs_df_3["indicator"] = [i[:-2] for i in mvs_df_3["year-indicator"]]
# drop year-indicator field 
mvs_df_1 = mvs_df_1.drop(["year-indicator"], axis = 1)
mvs_df_2 = mvs_df_2.drop(["year-indicator"], axis = 1)
mvs_df_3 = mvs_df_3.drop(["year-indicator"], axis = 1)
# ended up with duplicates because of NA values. Drop those 
mvs_df_1.dropna(subset = ["value"], inplace = True)
mvs_df_2.dropna(subset = ["value"], inplace = True)
mvs_df_3.dropna(subset = ["value"], inplace = True)
# it also seems like there are some indicators where the API failed to pull data, resulting in NAs in CSA2010 
mvs_df_1.dropna(subset = ["CSA"], inplace = True)
mvs_df_2.dropna(subset = ["CSA"], inplace = True)
mvs_df_3.dropna(subset = ["CSA"], inplace = True)



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (371) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# UNUSED 
##### pivot again, so that columns are years (not currently in use)
index_cols_pivotlonger = ["CSA", "indicator"]
mvs_pivot_1 = mvs_df_1.pivot(index = index_cols_pivotlonger,columns = "year", values = "value").reset_index()
mvs_pivot_2 = mvs_df_2.pivot(index = index_cols_pivotlonger,columns = "year", values = "value").reset_index()
mvs_pivot_3 = mvs_df_3.pivot(index = index_cols_pivotlonger,columns = "year", values = "value").reset_index()

### small data fix - remove some placeholder rows from mvs3 
mvs_df_3 = mvs_df_3.query("indicator != 'CSA2010'")
# export, for posterity
mvs_pivot_1.to_csv("modern_vital_signs_pivot_1.csv", index = False)
mvs_pivot_2.to_csv("modern_vital_signs_pivot_2.csv", index = False)
mvs_pivot_3.to_csv("modern_vital_signs_pivot_3.csv", index = False)

#### Compile into one dataframe

In [129]:
### concatenate into one big dataframe of vital signs 
vs = pd.concat([hvs, mvs_df_1,mvs_df_2,mvs_df_3])

In [130]:
## clean some names 
# remove asterisk from CSA names
# correct some spellings 
# unify some names that may be abbreviated
vs["CSA"] = vs.CSA.str.replace("*","", regex = False)
vs["CSA"] = vs.CSA.str.replace("Edmonson","Edmondson", regex = False)
vs["CSA"] = vs.CSA.str.replace("Falstaff","Fallstaff", regex = False) # really not sure which is right, but BNIA uses Fallstaff in modern communications
vs["CSA"] = vs.CSA.str.replace("Mt. Washington","Mount Washington", regex = False)
vs["CSA"] = vs.CSA.str.replace("Mt. Winans","Mount Winans", regex = False)
# make indicator lowercase (for uniformity)
vs["indicator"] = vs.indicator.str.lower()

Things that need to get cleaned up: 
- anything with a * 
- Edmonson Village vs Edmondson Village
- Glen-Fallstaff vs. Glen-Falstaff
- Jonestown/Oldtown vs. Oldtown / Middle East
- Washington Village vs. Washington Village/Pigtown
- Westport/Mount Winans/Lakeland vs Westport/Mt. Winans/Lakeland
- Perkins/Middle East vs Oldtown/Middle East
- 'Medfield/Hampden/Woodberry', vs 'Medfield/Hampden/Woodberry/Remington'
-  'Mount Washington/Coldspring','Mt. Washington/Coldspring'

Additionally: 
- we don't have year for hhsize 

In [136]:
# one-off issue: hhsize API returns a dataframe that does not include year, so we need to do some correcting 
# rename the indicator
vs["indicator"]  = vs.indicator.replace("hhsi","hhsize")
# replace the year value with unknown 
vs["year"] = vs.year.replace("20ze","uknown")

In [137]:
# export for posterity 
vs.to_csv("full_vital_signs.csv", index = False)

In [153]:
## create an info dataframe of indicator, years available, description, and category 
# function for a sorted set 
def sort_set(series):
  x = set(series)
  return sorted(x)

# get dataframe of indicators and years available for each (sorted)
info = vs[["indicator","year"]].groupby(["indicator"])["year"].apply(sort_set).reset_index()
# grab the info from the api DF
indicator_desc = api_df.rename(columns = {"Indicator":"indicator_description","shortname_noX":"indicator","Section":"category"})[["indicator_description","indicator","category"]]
info = info.merge(indicator_desc, how = "left")
info

,indicator,year,indicator_description,category
0,aastud,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",Percent of Students that are African American ...,Education and Youth
1,abse,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",Percent of 1st-5th Grade Students that are Chr...,Education and Youth
2,abshs,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",Percent of 9th-12th Grade Students that are Ch...,Education and Youth
3,absmd,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",Percent of 6th-8th Grade Students that are Chr...,Education and Youth
4,affordm,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",Affordability Index - Mortgage,Housing and Community Development
...,...,...,...,...
159,viol,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...","Violent Crime Rate per 1,000 Residents",Crime and Safety
160,voted,"[2010, 2012, 2014, 2016, 2018]",Percent Population (Over the age of 18) Who Vo...,Sustainability
161,walked,"[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201...",Percent of Population that Walks to Work,Sustainability
162,wlksc,"[2011, 2017]",Walk Score,Sustainability


In [154]:
# export 
info.to_csv("indicator_info.csv", index = False)

In [155]:
# how many unique indicators did I end up with? 
len(set(vs.indicator)) # great

164

In [156]:
indicator_list = list(set(vs.indicator))
indicator_list.sort()

In [157]:
indicator_list

['aastud',
 'abse',
 'abshs',
 'absmd',
 'affordm',
 'affordr',
 'age0-18_',
 'age18_',
 'age24_',
 'age45-64_',
 'age5_',
 'age64_',
 'age65_',
 'arrest',
 'artevnt',
 'bahigher',
 'baltvac',
 'banks',
 'birthwt',
 'biz1_',
 'biz2_',
 'biz4_',
 'bkln',
 'business_50-99_emp',
 'busload',
 'caracc',
 'caslt',
 'clogged',
 'cmos',
 'community_dev_corporations',
 'community_gardens',
 'comp',
 'compl',
 'comprop',
 'constper',
 'crehab',
 'crime',
 'dirtyst',
 'dom',
 'domvio',
 'drop',
 'eattend',
 'ebll',
 'elheat',
 'empl',
 'fam',
 'familiesrelatedkids',
 'farms',
 'fastfd',
 'female',
 'femhhs',
 'fore',
 'gunhom',
 'hazardous_waste_sites_count',
 'hcvhouse',
 'heatgas',
 'hfai',
 'hh25inc',
 'hh40inc',
 'hh60inc',
 'hh75inc',
 'hhchpov',
 'hhm75',
 'hhpov',
 'hhs',
 'hhsize',
 'historical_buildings',
 'hs_degree_only_25plus',
 'hsaalg',
 'hsabio',
 'hsaeng',
 'hsagov',
 'hsattend',
 'hsdipl',
 'hstud',
 'individuals_below_povline',
 'juvarr',
 'juvdrug',
 'juvviol',
 'kra',
 'leadte

### Grabbing Geometry

In [108]:
### pull a geodataframe using Colin's functions 

# functions
def getGDFfromURL(url, layer=0):
    #GDF stands for GeoDataFrame; this is the innermost function called by getGDF
    tail = "/"+str(layer)+"/query?where=1%3D1&outFields=*&outSR=4326&f=json" #worked this out
    url+=tail
    print(url)
    gdf = gpd.read_file(url) #GeoPandas has a built in function to read APIs given right URL
    return gdf

def getGDF(shortname, level=0):
    #This is outermost function called by user; it calls getGDFfromURL
    url = api_df.loc[shortname, "API"]
    return getGDFfromURL(url, level)

# using HH size as our sample GDF (since I had questions about this indicator anyway)
hhsizeXX = getGDF("hhsizeXX")
hhsizeXX

https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Hhsize/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json


,OBJECTID,CSA2010,Hhsize,Shape__Area,Shape__Length,geometry
0,1,Allendale/Irvington/S. Hilton,2.64,6.377046e+07,38770.165571,"POLYGON ((-76.65726 39.27600, -76.65726 39.276..."
1,2,Beechfield/Ten Hills/West Hills,2.40,4.788253e+07,37524.950533,"POLYGON ((-76.69479 39.30201, -76.69465 39.301..."
2,3,Belair-Edison,2.90,4.495003e+07,31307.314843,"POLYGON ((-76.56761 39.32636, -76.56746 39.326..."
3,4,Brooklyn/Curtis Bay/Hawkins Point,2.61,1.760777e+08,150987.703639,"MULTIPOLYGON (((-76.58867 39.21283, -76.58824 ..."
4,5,Canton,1.86,1.540854e+07,23338.611948,"POLYGON ((-76.57140 39.28441, -76.57138 39.284..."
5,6,Cedonia/Frankford,2.54,7.154134e+07,39962.549835,"POLYGON ((-76.52972 39.33113, -76.53005 39.331..."
6,7,Cherry Hill,2.57,2.914418e+07,22305.302067,"POLYGON ((-76.61525 39.24102, -76.61530 39.241..."
7,8,Chinquapin Park/Belvedere,2.26,2.102259e+07,21705.411630,"POLYGON ((-76.59491 39.35406, -76.59507 39.354..."
8,9,Claremont/Armistead,2.41,6.124377e+07,40104.415980,"POLYGON ((-76.52975 39.31020, -76.52990 39.310..."
9,10,Clifton-Berea,2.83,2.328523e+07,24022.764621,"POLYGON ((-76.58449 39.31491, -76.58249 39.314..."


In [161]:
### get a dataframe of neighborhoods and geometries 
geos = hhsizeXX[['CSA2010','Shape__Area', 'Shape__Length','geometry']].copy()

# correct CSA names to align with vs dataframe
geos = geos.rename(columns = {"CSA2010":"CSA"})
geos["CSA"] = geos.CSA.str.replace("*","", regex = False)
geos["CSA"] = geos.CSA.str.replace("Edmonson","Edmondson", regex = False)
geos["CSA"] = geos.CSA.str.replace("Falstaff","Fallstaff", regex = False) # really not sure which is right, but BNIA uses Fallstaff in modern communications
geos["CSA"] = geos.CSA.str.replace("Mt. Washington","Mount Washington", regex = False)
geos["CSA"] = geos.CSA.str.replace("Mt. Winans","Mount Winans", regex = False)

In [167]:
# side note - get list of CSAs to check for duplicates (include which years are listed with which name)
csa_by_year = vs[["CSA","year"]].groupby(["CSA"])["year"].apply(sort_set).reset_index()
# write to csv to examine in Excel 
csa_by_year.to_csv("csa_names.csv", index = False)